# 📊 Análisis del Dataset Aumentado

Este notebook analiza la calidad y características del dataset aumentado para determinar si es útil para mejorar el modelo.

## Objetivos:
1. Analizar balance de clases
2. Verificar calidad del texto aumentado
3. Comparar estadísticas original vs aumentado
4. Identificar problemas potenciales
5. Recomendar acciones


## 1. Importar librerías y cargar datasets


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Cargar datasets
df_original = pd.read_csv('../data/raw/youtoxic_english_1000.csv')
df_augmented = pd.read_csv('../data/processed/youtoxic_english_1000_augmented.csv')

print("✅ Datasets cargados:")
print(f"   Original: {len(df_original)} ejemplos")
print(f"   Aumentado: {len(df_augmented)} ejemplos")
print(f"   Incremento: {len(df_augmented) - len(df_original)} ejemplos ({((len(df_augmented) - len(df_original))/len(df_original)*100):.1f}%)")


## 2. Análisis de Balance de Clases y Calidad


In [ ]:
print("📊 ANÁLISIS COMPLETO DEL DATASET AUMENTADO:")
print("="*60)

# Balance de clases
orig_toxic = df_original['IsToxic'].sum()
orig_not_toxic = len(df_original) - orig_toxic
orig_toxic_pct = (orig_toxic / len(df_original)) * 100

aug_toxic = df_augmented['IsToxic'].sum()
aug_not_toxic = len(df_augmented) - aug_toxic
aug_toxic_pct = (aug_toxic / len(df_augmented)) * 100

aug_only = df_augmented[df_augmented['_augmented'] == True] if '_augmented' in df_augmented.columns else pd.DataFrame()

print(f"\n📈 Balance de Clases:")
print(f"   Original: {orig_toxic} tóxicos ({orig_toxic_pct:.1f}%), {orig_not_toxic} no tóxicos ({100-orig_toxic_pct:.1f}%)")
print(f"   Aumentado: {aug_toxic} tóxicos ({aug_toxic_pct:.1f}%), {aug_not_toxic} no tóxicos ({100-aug_toxic_pct:.1f}%)")

if len(aug_only) > 0:
    aug_only_toxic = aug_only['IsToxic'].sum()
    aug_only_pct = (aug_only_toxic / len(aug_only)) * 100
    print(f"   Solo aumentados: {aug_only_toxic} tóxicos ({aug_only_pct:.1f}%) de {len(aug_only)} ejemplos")

balance_change = abs(aug_toxic_pct - orig_toxic_pct)
print(f"\n   Cambio en balance: {balance_change:.1f}%")
if balance_change < 2:
    print("   ✅ Balance se mantiene correctamente")
else:
    print("   ⚠️  Balance cambió significativamente")

# Calidad del texto
print(f"\n📏 Calidad del Texto:")
orig_lengths = df_original['Text'].str.len()
aug_lengths = df_augmented['Text'].str.len()
print(f"   Longitud promedio - Original: {orig_lengths.mean():.1f}, Aumentado: {aug_lengths.mean():.1f}")

if len(aug_only) > 0:
    aug_only_lengths = aug_only['Text'].str.len()
    print(f"   Longitud promedio - Solo aumentados: {aug_only_lengths.mean():.1f}")

# Duplicados
duplicates = df_augmented.duplicated(subset=['Text'], keep=False)
print(f"\n🔍 Duplicados: {duplicates.sum()} textos duplicados ({duplicates.sum()/len(df_augmented)*100:.1f}%)")
if duplicates.sum() < len(df_augmented) * 0.05:
    print("   ✅ Pocos duplicados, dataset limpio")
else:
    print("   ⚠️  Muchos duplicados, considerar limpiar")


## 3. Verificar Resultados del Modelo Aumentado


In [ ]:
# Verificar si el modelo aumentado se guardó (significa que mejoró)
import pickle
model_path = Path('../models/augmented/svm_augmented_model_info.pkl')

if model_path.exists():
    with open(model_path, 'rb') as f:
        model_info = pickle.load(f)
    
    print("✅ MODELO AUMENTADO ENCONTRADO (Mejoró el rendimiento):")
    print("="*60)
    print(f"   F1-Score (Test): {model_info.get('test_f1', 'N/A'):.4f}")
    print(f"   Accuracy: {model_info.get('test_accuracy', 'N/A'):.4f}")
    print(f"   Overfitting: {model_info.get('overfitting', 'N/A'):.2f}%")
    print(f"   Dataset size: {model_info.get('dataset_size', 'N/A')}")
    print(f"   Método: {model_info.get('augmentation_method', 'N/A')}")
    
    print("\n💡 RECOMENDACIÓN: Usar modelo aumentado en producción")
else:
    print("ℹ️  Modelo aumentado no se guardó (no mejoró significativamente)")
    print("   El modelo original sigue siendo el mejor")


## 4. Decisión Final y Próximos Pasos


In [ ]:
print("="*60)
print("📌 DECISIÓN Y PRÓXIMOS PASOS:")
print("="*60)

if model_path.exists():
    print("\n✅ EL DATASET AUMENTADO MEJORÓ EL MODELO")
    print("\n📋 Acciones recomendadas:")
    print("   1. ✅ Dataset aumentado es de buena calidad")
    print("   2. ✅ Modelo aumentado tiene mejor rendimiento")
    print("   3. 🔄 OPCIONAL: Actualizar API para usar modelo aumentado")
    print("      - Cambiar ruta del modelo en backend/src/api/predict.py")
    print("      - De: models/optimized/best_optimized_model.pkl")
    print("      - A: models/augmented/svm_augmented_model.pkl")
    print("   4. 📝 Documentar en README que se usa dataset aumentado")
    print("\n⚠️  NOTA: Si actualizas el modelo en producción, verifica que:")
    print("   - El overfitting sigue siendo bajo (<5%)")
    print("   - Las métricas mejoraron significativamente")
    print("   - El modelo funciona correctamente en la API")
else:
    print("\n➖ EL DATASET AUMENTADO NO MEJORÓ SIGNIFICATIVAMENTE")
    print("\n📋 Acciones recomendadas:")
    print("   1. ✅ Mantener modelo original (mejor rendimiento)")
    print("   2. ℹ️  Dataset aumentado puede usarse para análisis adicionales")
    print("   3. 🔄 OPCIONAL: Probar otras técnicas de augmentation")
    print("      - Traducción y back-translation")
    print("      - Parafraseo")
    print("      - Aumentar solo clase minoritaria")
    print("\n💡 CONCLUSIÓN: El dataset original es suficiente para este proyecto")
